In [8]:
# импортируем Pandas и Numpy
import pandas as pd
import numpy as np

# загружаем данные из csv
df = pd.read_csv('sf-salaries/Salaries/Salaries.csv', header = 0)

In [10]:
# Первые 5 строк датасета
df.head()

,Id,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year,Notes,Agency,Status
0,1,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,167411,0,400184,NaN,567595.43,567595.43,2011,NaN,San Francisco,NaN
1,2,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),155966,245132,137811,NaN,538909.28,538909.28,2011,NaN,San Francisco,NaN
2,3,ALBERT PARDINI,CAPTAIN III (POLICE DEPARTMENT),212739,106088,16452.6,NaN,335279.91,335279.91,2011,NaN,San Francisco,NaN
3,4,CHRISTOPHER CHONG,WIRE ROPE CABLE MAINTENANCE MECHANIC,77916,56120.7,198307,NaN,332343.61,332343.61,2011,NaN,San Francisco,NaN
4,5,PATRICK GARDNER,"DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)",134402,9737,182235,NaN,326373.19,326373.19,2011,NaN,San Francisco,NaN


In [11]:
#  Размер таблицы: 13 колонок и 148654 строк
print(df.shape)

(148654, 13)


In [13]:
# Информация по датасету 
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148654 entries, 0 to 148653
Data columns (total 13 columns):
Id                  148654 non-null int64
EmployeeName        148654 non-null object
JobTitle            148654 non-null object
BasePay             148049 non-null object
OvertimePay         148654 non-null object
OtherPay            148654 non-null object
Benefits            112495 non-null object
TotalPay            148654 non-null float64
TotalPayBenefits    148654 non-null float64
Year                148654 non-null int64
Notes               0 non-null float64
Agency              148654 non-null object
Status              38119 non-null object
dtypes: float64(3), int64(2), object(8)
memory usage: 14.7+ MB
None


In [ ]:
# Есть интересные колонки с зарплатами, но с типом object
# Как оказалось, в них присутствуют данные с пометкой 'Not Provided', т.е. не указано. 
# Тут можно поступить так: либо удалить эти строки, либо делать анализ по итоговой сумме дохода (вместе с бонусами,овертаймами и т.д.)
# Я проведу анализ по TotalPayBenefits - т.е. общая сумма+бенефиты

# Если решить удалять строки с пустыми данными, то потом
# необходимо изменить тип колонок с зарплатами BasePay OvertimePay OtherPay Benefits на тип float, 
# иначе по ним нельзя будет считать статистические показатели

df['BasePay'] = df['BasePay'].astype('float64')
df['OvertimePay'] = df['OvertimePay'].astype('float64')
df['OtherPay'] = df['OtherPay'].astype('float64')
df['Benefits'] = df['Benefits'].astype('float64')

In [18]:
# основные статистические показатели по числовым столбцам
df.describe()

,Id,TotalPay,TotalPayBenefits,Year,Notes
count,148654.000000,148654.000000,148654.000000,148654.000000,0.0
mean,74327.500000,74768.321972,93692.554811,2012.522643,NaN
std,42912.857795,50517.005274,62793.533483,1.117538,NaN
min,1.000000,-618.130000,-618.130000,2011.000000,NaN
25%,37164.250000,36168.995000,44065.650000,2012.000000,NaN
50%,74327.500000,71426.610000,92404.090000,2013.000000,NaN
75%,111490.750000,105839.135000,132876.450000,2014.000000,NaN
max,148654.000000,567595.430000,567595.430000,2014.000000,NaN


In [22]:
# меняем тип на object у столбцов Id, Year, Notes
df['Id'] = df['Id'].astype('object')
df['Year'] = df['Year'].astype('object')
df['Notes'] = df['Notes'].astype('object')

In [23]:
# основные статистические показатели по числовым столбцам
df.describe()

,TotalPay,TotalPayBenefits
count,148654.000000,148654.000000
mean,74768.321972,93692.554811
std,50517.005274,62793.533483
min,-618.130000,-618.130000
25%,36168.995000,44065.650000
50%,71426.610000,92404.090000
75%,105839.135000,132876.450000
max,567595.430000,567595.430000


In [24]:
# кол-во работников по должностям
df['JobTitle'].value_counts()

Transit Operator                  7036
Special Nurse                     4389
Registered Nurse                  3736
Public Svc Aide-Public Works      2518
Police Officer 3                  2421
                                  ... 
MOBILE EQUIPMENT SUPERVISOR          1
Auto Body&Fender Wrk Asst Sprv       1
ROOFER SUPERVISOR I                  1
Special Assistant 18                 1
SOFT FLOOR COVERER SUPERVISOR        1
Name: JobTitle, Length: 2159, dtype: int64

In [27]:
# в столбце TotalPay оказались нулевые значения, удаляем их и смотрим еще раз стат.показатели
df = df.drop(np.where(df['TotalPay'] <= 0)[0])

In [28]:
# основные статистические показатели по числовым столбцам
df.describe()

,TotalPay,TotalPayBenefits
count,148285.000000,148285.000000
mean,74954.383468,93909.164657
std,50441.766219,62720.972790
min,0.300000,0.300000
25%,36679.200000,44692.040000
50%,71553.000000,92572.300000
75%,105967.060000,133004.190000
max,567595.430000,567595.430000


In [51]:
# Создадим фрейм из первых 5 самых популярных профессиях и проведем анализ
df1 = pd.DataFrame()
df1 = df.loc[df['JobTitle'].isin(['Transit Operator', 'Special Nurse', 'Registered Nurse', 'Public Svc Aide-Public Works', 'Police Officer 3'])]
df1.describe()

,TotalPay,TotalPayBenefits
count,20047.000000,20047.000000
mean,68861.999387,89987.411770
std,51966.528733,66744.634062
min,1.770000,1.910000
25%,15456.290000,19097.440000
50%,70460.240000,99623.200000
75%,112981.935000,146284.370000
max,267902.740000,301749.810000


In [53]:
# статистические показатели по 5 самым поп.профессиям
columns_to_show = ['TotalPay', 'TotalPayBenefits']
df1.groupby(['JobTitle'])[columns_to_show].describe()

TotalPay                                      \
                                count           mean           std    min   
JobTitle                                                                    
Police Officer 3               2417.0  135130.154278  38039.037976  30.78   
Public Svc Aide-Public Works   2516.0    8248.861367   6588.894105  51.95   
Registered Nurse               3734.0  115762.566261  36819.068033   6.00   
Special Nurse                  4386.0   33184.617305  35564.440348   3.28   
Transit Operator               6994.0   65099.746884  29702.406888   1.77   

                                                                               \
                                      25%         50%          75%        max   
JobTitle                                                                        
Police Officer 3              127574.4100  137836.900  153915.4000  267902.74   
Public Svc Aide-Public Works    2947.5675    7198.735   11347.6100   44772.87   
Registered Nurse              103719.8425  124394.330  138304.0025  199600.46   
Special Nurse                   6417.9475   19462.855   48454.8500  179203.29   
Transit Operator               47515.0600   72784.805   84516.2925  157310.65   

                             TotalPayBenefits                               \
                                        count           mean           std   
JobTitle                                                                     
Police Officer 3                       2417.0  168010.924857  46562.515197   
Public Svc Aide-Public Works           2516.0    9624.696363   9542.585629   
Registered Nurse                       3734.0  151729.555857  47612.675520   
Special Nurse                          4386.0   36395.026947  37088.872320   
Transit Operator                       6994.0   92578.317496  40550.843951   

                                                                          \
                                min         25%         50%          75%   
JobTitle                                                                   
Police Officer 3              33.22  162701.970  174456.590  190655.5300   
Public Svc Aide-Public Works  52.47    3096.915    7465.165   11783.8275   
Registered Nurse               7.24  138162.945  164742.575  180866.6425   
Special Nurse                  4.16    7602.845   22775.180   54624.8075   
Transit Operator               1.91   69652.735  106272.350  119408.8900   

                                         
                                    max  
JobTitle                                 
Police Officer 3              301749.81  
Public Svc Aide-Public Works   66981.05  
Registered Nurse              253476.57  
Special Nurse                 182019.76  
Transit Operator              197206.13

In [59]:
# Как с годами менялись зп этих 5-ти выбранных профессий
df1.pivot_table(['TotalPay', 'TotalPayBenefits'], ['JobTitle', 'Year'], aggfunc='mean')

TotalPay  TotalPayBenefits
JobTitle                     Year                                 
Police Officer 3             2012  129077.159861     159810.206563
                             2013  140976.929870     174793.391364
                             2014  136059.617842     170390.134815
Public Svc Aide-Public Works 2012    7980.988223      10000.115168
                             2013    7329.612075       8139.835198
                             2014    9328.364798      10712.209705
Registered Nurse             2012  109683.024504     148483.138039
                             2013  120775.187024     155096.034293
                             2014  117023.096645     151715.116759
Special Nurse                2012   32766.103509      37314.407358
                             2013   33250.277945      35660.368042
                             2014   33535.730508      36201.727364
Transit Operator             2012   65757.454317      96027.714676
                             2013   68499.271170      95443.400256
                             2014   61392.559673      86818.427104